In [75]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [76]:
url = 'https://habr.com/ru/post/346198/'
page = requests.get(url)

In [77]:
df = pd.DataFrame(columns=['title', 'text', 'year', 'tags', 'votes', 'views', 'bookmarks', 'comments'])

In [79]:
soup = BeautifulSoup(page.text, 'html5lib')

In [80]:
text = soup.find("div",
    class_="post__text post__text-html").text

In [166]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

In [179]:
def isvotes(tag):
    return tag.has_attr('class') and tag.has_attr('onclick') and not tag.has_attr('href') and not tag.has_attr('type')

In [229]:
def document(id):
    page = requests.get('https://habr.com/ru/post/' + str(id) + '/')
    soup = BeautifulSoup(page.text, 'html5lib')
    info = []
    if soup.find("span", class_= "post__title-text"):
        #title
        info.append(soup.find("span", class_= "post__title-text").text)
        #text
        text = soup.find("div", class_="post__text post__text-html").text
        info.append(cleanhtml(text).split())
        #year
        info.append(int(soup.find("span", class_= "post__time").text.split()[2]))
        #tags
        tagsshit = soup.find_all("a", class_="inline-list__item-link hub-link")
        tags = []
        for i in range(len(tagsshit)):
            tags.append(tagsshit[i].text)
        info.append(tags)
        #votes
        info.append(int(soup.find(isvotes).text))
        #views
        a = soup.find("div", class_="post-stats__views").text.strip()
        if 'k' in a:
            views = float(a[:-1].replace(',', '.')) * 1000
        else:
            views = int(a)
        info.append(views)
        #bookmarks
        info.append(int(soup.find("span", class_="bookmark__counter js-favs_count").text))
        #comments
        comment1 = soup.find_all("div", class_="comment__message")
        comments = []
        for i in range(len(comment1)):
            comments.append(comment1[i].text)
        info.append(comments)
    else:
        info.append('NaN')
    print(id)
    return info
    

In [230]:
%time a = list(map(document, np.arange(346198,346248)))

346198
346199
346200
346201
346202
346203
346204
346205
346206
346207
346208
346209
346210
346211
346212
346213
346214
346215
346216
346217
346218
346219
346220
346221
346222
346223
346224
346225
346226
346227
346228
346229
346230
346231
346232
346233
346234
346235
346236
346237
346238
346239
346240
346241
346242
346243
346244
346245
346246
346247
Wall time: 32 s


In [219]:
df = pd.DataFrame(a, columns=['title', 'text', 'year', 'tags', 'votes', 'views', 'bookmarks', 'comments'])
df.dropna(inplace=True)

In [228]:
for i in df['tags']:
    if 'Платежные системы' in i:
        print(i)